In [1]:
#####################################################################
## Load All Game Stats
import os, pickle
stats = dict()
for filename in os.listdir('PlayerPickles/FootballStats/'):
    player = filename.replace('_game_stats.pkl', '')
    with open('PlayerPickles/FootballStats/' + filename, 'rb') as stats_pickle:
        stats[player] = pickle.load(stats_pickle)

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
import pickle

# Get player position
FOOTBALL_FANTASY_POINTS_URL_FORMAT_STRING = 'https://www.fantasypros.com/nfl/projections/{0}.php'
def generate_football_reference_url(player_name):
    name_url_safe = player_name.replace('_', '-')
    
    if name_url_safe == 'najee-harris':
        name_url_safe = 'najee-harris-rb'
    elif name_url_safe == 'dj-moore':
        name_url_safe = 'dj-moore-wr'
    elif name_url_safe == 'mike-williams':
        name_url_safe = 'mike-williams-wr'
    elif name_url_safe == 'josh-allen':
        name_url_safe = 'josh-allen-qb'
    elif name_url_safe == 'juju-smithschuster':
        name_url_safe = 'juju-smith-schuster'
    elif name_url_safe == 'clyde-edwardshelaire':
        name_url_safe = 'clyde-edwards-helaire'
    elif name_url_safe == 'amonra-st-brown':
        name_url_safe = 'amonra-stbrown'
    elif name_url_safe == 'elijah-mitchell':
        name_url_safe = 'elijah-mitchell-rb'
    elif name_url_safe == 'damien-harris':
        name_url_safe = 'damien-harris-rb'
    elif name_url_safe == 'donovan-peoplesjones':
        name_url_safe = 'donovan-peoples-jones'
    elif name_url_safe == 'jeff-wilson':
        name_url_safe = 'jeffery-wilson'
    elif name_url_safe == 'ronald-jones':
        name_url_safe = 'ronald-jones-ii'
    elif name_url_safe == 'robbie-anderson':
        name_url_safe = 'robby-anderson'
    elif name_url_safe == 'odell-beckham-jr':
        name_url_safe = 'odell-beckham'
            
        
    return FOOTBALL_FANTASY_POINTS_URL_FORMAT_STRING.format(name_url_safe)

positions = dict()
for player in stats.keys():
    fp_res = requests.get(generate_football_reference_url(player))
    fp_soup = BeautifulSoup(fp_res.content,'lxml')
    positions[player] = fp_soup.find_all('h1')[1].parent.find_all('h2')[0].text[:2]

In [3]:
positions

{'darnell_mooney': 'WR',
 'rashaad_penny': 'RB',
 'keenan_allen': 'WR',
 'nico_collins': 'WR',
 'hunter_renfrow': 'WR',
 'diontae_johnson': 'WR',
 'james_proche': 'WR',
 'kadarius_toney': 'WR',
 'noah_fant': 'TE',
 'robert_woods': 'WR',
 'javonte_williams': 'RB',
 'tee_higgins': 'WR',
 'davante_adams': 'WR',
 'marlon_mack': 'RB',
 'james_robinson': 'RB',
 'kyle_rudolph': 'TE',
 'dontrell_hilliard': 'RB',
 'david_njoku': 'TE',
 'robert_tonyan': 'TE',
 'chase_edmonds': 'RB',
 'jalen_hurts': 'QB',
 'darren_waller': 'TE',
 'ameer_abdullah': 'RB',
 'jameis_winston': 'QB',
 'alvin_kamara': 'RB',
 'kareem_hunt': 'RB',
 'jonathan_taylor': 'RB',
 'eno_benjamin': 'RB',
 'dandre_swift': 'RB',
 'melvin_gordon': 'RB',
 'mark_ingram': 'RB',
 'mecole_hardman': 'WR',
 'michael_pittman': 'WR',
 'michael_thomas': 'S ',
 'trey_sermon': 'RB',
 'donovan_peoplesjones': 'WR',
 'jerick_mckinnon': 'RB',
 'sterling_shepard': 'WR',
 'najee_harris': 'RB',
 'jakobi_meyers': 'WR',
 'julio_jones': 'WR',
 'mark_andre

In [4]:
#####################################################################
## Helper Methods
def make_df_from_rows(data, col_labels):
    '''
        Adds matrix rows to a new DF with given column labels.
        If dict, keys are ignored and values added to axis=0.
    '''
    converted_df = pd.DataFrame([], columns=col_labels)
    if type(data) == list:
        pass
    elif type(data) == dict:
        for key, value in data.items():
            filtered_row = value[[lbl for lbl in col_labels]]
            converted_df = converted_df.append(filtered_row, ignore_index=True)
    
    return converted_df

In [5]:
stats

{'darnell_mooney':     Projected Pts  Actual Pts   +/- Exceeded Expectations?
 0             8.9        12.6   3.7                    Yes
 1             8.3        12.9   4.6                    Yes
 2             8.7         5.9  -2.8                     No
 3             9.1         6.6  -2.5                     No
 4             8.9         1.9  -7.0                     No
 5             9.9         2.7  -7.2                     No
 6            10.1        12.5   2.4                    Yes
 7             8.9        18.1   9.2                    Yes
 8             6.4        17.6  11.2                    Yes
 9             6.2         6.4   0.2                    Yes
 10            6.7         3.9  -2.8                     No
 11            6.6        10.5   3.9                    Yes
 12            6.9         3.5  -3.4                     No
 13            6.5        13.5   7.0                    Yes
 14            6.6         0.9  -5.7                     No
 15            6.5    

In [6]:
analysis_stats = dict()
analysis_std = dict()
for player, df_stats in stats.items():
    analysis_stats[player] = df_stats.mean()
    analysis_std[player] = df_stats.std()
    series_expectation = df_stats.apply(lambda x: True if x['Exceeded Expectations?'] == 'Yes' else False , axis=1)
    analysis_stats[player]['Exceeded Expectations?'] = float((series_expectation == True).sum()) / len(series_expectation)

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stat

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_st

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stats.std()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_stats[player] = df_stats.mean()
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/1345911946.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  analysis_std[player] = df_stat

In [7]:
cum_stats = make_df_from_rows(analysis_stats, list(analysis_stats['christian_mccaffrey'].index))
cum_std = make_df_from_rows(analysis_std, ['Projected Pts', 'Actual Pts', '+/-'])
cum_stats.index = analysis_stats.keys()
cum_std.index = analysis_std.keys()

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  converted_df = converted_df.append(filtered_row, ignore_index=True)
/var/folders/bq/h7ljfy0x5jng5l25x9j79_0h0000gn/T/ipykernel_1653/2847935649.py:14: FutureWarning: The frame.append method is deprecated and will b

In [8]:
cum_std.sort_values(by='Actual Pts', ascending=False)

,Projected Pts,Actual Pts,+/-
derrick_henry,1.653514,12.711799,13.724951
rashaad_penny,3.355278,12.137367,9.755568
jonathan_taylor,2.432727,11.012747,11.383034
jamarr_chase,1.524192,10.94087,11.288421
lamar_jackson,1.360119,10.842663,10.51652
...,...,...,...
joshua_kelley,1.264629,1.17108,1.819537
cam_akers,3.394113,0.919239,2.474874
curtis_samuel,0.732803,0.879204,1.050238
jk_dobbins,NaN,NaN,NaN


In [9]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(cum_stats)
scaled_stats = pd.DataFrame(scaled, columns=cum_stats.columns, index=cum_stats.index)

std_scaler = MinMaxScaler()
scaled = std_scaler.fit_transform(cum_std)
scaled_std = pd.DataFrame(scaled, columns = cum_std.columns, index=cum_std.index)

In [10]:
scaled_std.sort_values(by='Actual Pts', ascending=False)

,Projected Pts,Actual Pts,+/-
derrick_henry,0.180247,1.000000,1.000000
rashaad_penny,0.365754,0.951453,0.686827
jonathan_taylor,0.265188,0.856409,0.815229
jamarr_chase,0.166150,0.850335,0.807764
lamar_jackson,0.148264,0.842035,0.746864
...,...,...,...
joshua_kelley,0.137855,0.024667,0.060696
cam_akers,0.369987,0.003383,0.112400
curtis_samuel,0.079882,0.000000,0.000000
jk_dobbins,NaN,NaN,NaN


In [11]:
qbs = []
rbs = []
wrs = []
tes = []
for player in scaled_stats.index:
    if positions[player] == 'QB':
        qbs.append(player)
    elif positions[player] == 'RB':
        rbs.append(player)
    elif positions[player] == 'WR':
        wrs.append(player)
    elif positions[player] == 'TE':
        tes.append(player)

In [12]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

source = ColumnDataSource(
        data=dict(
            x=scaled_stats.loc[wrs]['Actual Pts'],
            y=scaled_std.loc[wrs]['Actual Pts'],
            desc=wrs,
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ]
    )

p = figure(plot_width=700, plot_height=700, tools=[hover],
           title="Mouse over the dots")

p.circle('x', 'y', size=10, source=source)
show(p)

Loading BokehJS ...

In [13]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

source = ColumnDataSource(
        data=dict(
            x=scaled_stats.loc[rbs]['Actual Pts'],
            y=scaled_stats.loc[rbs]['Exceeded Expectations?'],
            desc=rbs,
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ]
    )

p = figure(plot_width=700, plot_height=700, tools=[hover],
           title="Mouse over the dots")

p.circle('x', 'y', size=10, source=source)
show(p)

Loading BokehJS ...

# Strategy

This year, it is pretty clear QBs are pretty close together, DO NOT DRAFT A QB until there's no more wrs/rbs that
I know will be year long keepers.

TE's are even a little better, can go late but will be struggling a little

If 1-3 ez RB, if 4-8 consider WR, some decent RBs in 9-14 range, but then quickly deteriorates with good WR options

WR's are a little harder than RB's this year on the elite level, but there are plenty of potential breakout WR's. Consider first-picking Justin Jefferson, Cooper will regress if I'm 3-6 on order, or one of the god WR's in the first two rounds. Then, focus 1/2 solid RB's when the WR pool gets thinner and then pick some fine WR sleepers later.

After I have a QB I intend on keeping, maybe consider looking at the rookies

In [14]:
# Taylor is up and coming, henry is safe, ekeler is safe, kupp is safe (wr is hard), najee is
# young, mixon is safe, jefferson and jamarr are up and coming (wr is hard), dalvin misses lots,
# deebo is old and coming off god season, kamara is suspended early
elite_top_12 = ['jonathan_taylor', 'derrick_henry', 'christian_mccaffrey', 'austin_ekeler',
                'cooper_kupp', 'najee_harris', 'joe_mixon', 'justin_jefferson', 'jamarr_chase',
                'dalvin_cook', 'deebo_samuel', 'alvin_kamara']
# Diggs had bad suppport in 2021, davante misses games, fournette is old, ceedee
# is a huge up and coming, tyreek is high tier not elite, chubb is NOT elite, mark andrews is
# THE BEST TE, swift is clear RB1, kelce is old but clear 2nd TE, akers missed last season,
# javonte is super young and unreliable, higgins is great but chase may become elite,
# James conner has upside, but may not be the guy espn thinks he is, WOULD TAKE MAHOMES OVER CONNER 
high_tier_13_to_26 = ['stefon_diggs', 'davante_adams', 'leonard_fournette', 'ceedee_lamb',
                      'tyreek_hill', 'nick_chubb', 'mark_andrews', 'dandre_swift',
                      'travis_kelce', 'cam_akers', 'javonte_williams', 'tee_higgins',
                      'james_conner', 'mike_evans']
# Evans is in a downtrend, allen and moore need to prove themselves, JONES IS OLD AND WILL,
# NOW BE SPLIT TIME, BREECE HALL IS A ROOKIE, AJ Brown is sleeper elite, williams is only good,
# pittman is only good, saquon will need to have a huge comeback season, mclaurin is nice, 
# sutton is also only nice, marquise is only relevant bc of early year suspension of other,
# diontae is okay, pitts is a superstar TE but nowhere near top 2, dobbins may struggle,
# MONTGOMERY is a nice late RB, elliott is worn but battle tested, josh allen is best QB bt
# not by much, waddle has more competition now, metcalf has no QB, JEUDY is A SLEEPER WITH RUSS WILSON,
nice_27_to_46 = ['dj_moore', 'keenan_allen', 'aaron_jones', 'aj_brown',
                 'mike_williams', 'michael_pittman', 'saquon_barkley', 'terry_mclaurin',
                'courtland_sutton', 'marquise_brown', 'diontae_johnson', 'kyle_pitts',
                'jk_dobbins', 'david_montgomery', 'ezekiel_elliott', 'josh_allen', 'jaylen_waddle',
                'jerry_jeudy']
# CHRIS GOODWIN IS INJURED, WILL BE A WR1 WHEN HES BACK, MAYBE GET HIM LATE IF IM SOLID
# Gabe davis is not good, needs big jump. josh jacobs was a big disappointment, cooks is a little
# underrated, amonra will have more competition now, mooney is a reliable WR3, herb kyler mahomes lamar
# are all QBs, no big advantage this year for QBs, mitchell doesn't get enough but he may now,
# devin is devin, may be a little better this year, SANDERS IS SLEEPER, CEH may be good,
# waller and kittle are solid TEs, robinson is just a flex, cooper may jump to WR2,
# thomas is risky but may be WR2, juju may have 1st seat upside on his team
# LOOK TO PICK ONE OF COOPER, THOMAS, OR JUJU AS A SLEEPER LATE ROUND WR,
# Thielen Smith and Bateman are just WR3/Flexes
# Etienne is a pretty solid rookie
support_47_to_70 = ['gabriel_davis', 'josh_jacobs', 'brandin_cooks', 'amonra_st_brown', 'darnell_mooney',
                    'chris_godwin', 'patrick_mahomes', 'justin_herbert', 'elijah_mitchell',
                    'devin_singletary', 'miles_sanders', 'clyde_edwardshelaire',
                    'darren_waller', 'george_kittle', 'lamar_jackson', 'kyler_murray', 'allen_robinson',
                    'amari_cooper', 'michael_thomas', 'juju_smithschuster', 'adam_thielen', 'devonta_smith',
                    'rashod_bateman']
# Everyone afterwards is just meh, consider tyler lockett

In [20]:
scaled_stats.loc[high_tier_13_to_26].sort_values(by='Actual Pts', ascending=False)

,Projected Pts,Actual Pts,+/-,Exceeded Expectations?
james_conner,0.398594,0.598819,0.752726,0.733333
nick_chubb,0.587070,0.567752,0.514399,0.428571
davante_adams,0.578851,0.562919,0.518070,0.375000
leonard_fournette,0.470994,0.542460,0.612922,0.500000
mike_evans,0.449030,0.479486,0.568358,0.437500
mark_andrews,0.389315,0.464688,0.617101,0.470588
dandre_swift,0.497825,0.459899,0.494117,0.538462
tyreek_hill,0.559415,0.444099,0.410100,0.294118
stefon_diggs,0.507906,0.436916,0.458210,0.352941
travis_kelce,0.494595,0.434463,0.469993,0.375000


In [16]:
scaled_stats.loc['james_conner']

Projected Pts             0.398594
Actual Pts                0.598819
+/-                       0.752726
Exceeded Expectations?    0.733333
Name: james_conner, dtype: float64

In [17]:
scaled_stats.loc['javonte_williams']

Projected Pts             0.376617
Actual Pts                0.387599
+/-                       0.547147
Exceeded Expectations?    0.411765
Name: javonte_williams, dtype: float64

In [18]:
pd.options.display.max_rows = 999
scaled_stats.sort_values(by='Actual Pts', ascending=False).head(186)

,Projected Pts,Actual Pts,+/-,Exceeded Expectations?
josh_allen,1.000000,1.000000,0.535705,0.411765
justin_herbert,0.864159,0.947091,0.625683,0.529412
tom_brady,0.881409,0.925784,0.583814,0.588235
kyler_murray,0.932678,0.902647,0.503031,0.500000
patrick_mahomes,0.952084,0.895858,0.474593,0.470588
derrick_henry,0.756019,0.891818,0.683024,0.500000
jalen_hurts,0.881904,0.871503,0.524315,0.533333
lamar_jackson,0.968995,0.858072,0.415193,0.250000
aaron_rodgers,0.808457,0.855444,0.586594,0.625000
dak_prescott,0.846640,0.840436,0.528846,0.500000


In [19]:
scaled_std.sort_values(by='Actual Pts', ascending=False)

,Projected Pts,Actual Pts,+/-
derrick_henry,0.180247,1.000000,1.000000
rashaad_penny,0.365754,0.951453,0.686827
jonathan_taylor,0.265188,0.856409,0.815229
jamarr_chase,0.166150,0.850335,0.807764
lamar_jackson,0.148264,0.842035,0.746864
tyreek_hill,0.103267,0.777045,0.705483
leonard_fournette,0.252250,0.707743,0.646184
trey_lance,1.000000,0.654753,0.554371
tom_brady,0.115753,0.652012,0.592226
dak_prescott,0.161336,0.650889,0.651004
